<a href="https://colab.research.google.com/github/chonholee/tutorial/blob/main/bigdata/BigDataII_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# いつものDriveマウント

from google.colab import drive
drive.mount('/content/drive/')
%cd '/content/drive/MyDrive/Lecture_BigData'

# スクレイピング

# 演習１

In [1]:
import requests
from bs4 import BeautifulSoup

In [36]:
url = 'https://chonholee.github.io/tutorial/index.html'
response = requests.get(url)

In [ ]:
# https://developer.mozilla.org/ja/docs/Web/HTTP/Status

response.status_code

In [ ]:
response.encoding #文字コード

In [ ]:
response.content #取得した内容

In [40]:
soup = BeautifulSoup(response.content, 'html.parser')

In [ ]:
soup

In [ ]:
soup.title

In [ ]:
soup.title.string

### HTML要素を取得する
### find_all() や select() などの使い方
https://gammasoft.jp/blog/difference-find-and-select-in-beautiful-soup-of-python/


In [ ]:
# ホームページのソースを見ながら、取得したい情報のタグを記入

result = soup.find_all("body")
result

In [ ]:
# ページ内の全てのリンクの名前とアドレスとを抽出する

for r in soup.find_all("a"):
    print(r.string, r['href'])

In [ ]:
# ページ内の全てのリンクアドレスをリストにする

links = [x.get('href') for x in soup.find_all('a')]
print(links)

# 演習2　テーブル

In [ ]:
url = 'https://chonholee.github.io/tutorial/bigdata/test-table.html'
r = requests.get(url)
soup = BeautifulSoup(r.content, 'html.parser')
soup

In [ ]:
result = soup.find_all('th')
for a in result:
    print(a)

### 2-2　取得したデータをData Frameとして保存する

In [ ]:
import pandas as pd
import numpy as np

th_list = []
td_list = []
for a in soup.find_all('th'):
    th_list.append(a.string)

for a in soup.find_all('td'):
    td_list.append(a.string)

df = pd.DataFrame(np.array([th_list, td_list]).T, columns=['項目', '値'])

df.head()


In [9]:
df.to_csv('data.csv', index=False)

# 演習３　画像

In [ ]:
r = requests.get('https://chonholee.github.io/tutorial/bigdata/test-image.html')
soup = BeautifulSoup(r.content, 'html.parser')
soup

In [ ]:
soup.find('img') # 一つ
soup.find_all('img') # 全部

In [ ]:
soup.find('img')['src']

In [13]:
from PIL import Image
import io

page_dir = 'https://chonholee.github.io/tutorial/bigdata/'

img_url = soup.find('img')['src']

img_binary = io.BytesIO(requests.get(page_dir+img_url).content) #画像のURLから画像をバイナリーデータに変換

img = Image.open(img_binary) #バイナリーデータを画像化

In [ ]:
img

### 3-2 imgタグすべてを取得して画像を保存する

In [80]:
img_tag_list = soup.find_all('img') #imgタグをすべて取得

for i, img_tag in enumerate(img_tag_list):
    img_url = img_tag['src'] #画像URL取得
    img_binary = io.BytesIO(requests.get(page_dir+img_url).content) #バイナリーデータ化
    img = Image.open(img_binary) #画像化
    img.save('{}.jpg'.format(i)) #保存

# 演習４　プロ野球データ

In [27]:
import requests
from bs4 import BeautifulSoup as BS

# HTMLを取得
url = 'https://baseball-data.com/stats/hitter2-all/tpa-2.html'
res = requests.get(url)
content = res.text

In [ ]:
content

## htmlコードを見て、取得したいデータのタグを検証する

In [29]:
# table 要素を取得
bs = BS(content, "lxml")  # parserの高速化　https://naruport.com/blog/2019/7/13/how-to-use-of-beautiful-soup-4/
tables = bs.find_all("table")
n_tables = len(tables)

In [30]:
tables

#抽出したテーブルが複数ある場合
if n_tables > 1:
    for i in range(n_tables):
        print(tables[i])            

In [ ]:
# table head 要素を取得 (存在する場合)
thead = table.find("thead")

# thead が存在する場合
if thead:
    tr = thead.find("tr")
    ths = tr.find_all("th")
    columns = [th.text for th in ths]    # pandas.DataFrame を意識
# thead が存在しない場合
else:
    columns = []

columns

In [ ]:
# table body 要素を取得
tbody = table.find("tbody")

# tr 要素を取得
trs = tbody.find_all("tr")

# 出力したい行データ
rows = [columns]

# td (th) 要素の値を読み込む
for tr in trs:
    row = [td.text for td in tr.find_all(["td"])]
    rows.append(row)

rows

In [33]:
# データを書き込む
import csv

filepath = "baseball_data.csv"

with open(filepath, "w") as f:
    writer = csv.writer(f, lineterminator="\n")
    writer.writerows(rows)